In [36]:
import numpy as np

# Simulación de una matriz de imagen en escala de grises de 10x10
np.random.seed(10)  # Semilla para reproducibilidad
image_matrix = np.random.randint(0, 256, size=(100, 100))
print(image_matrix)

[[  9 125 228 ... 205  88 143]
 [253 134 246 ... 161  95 171]
 [216 224 229 ...  44   9  50]
 ...
 [ 94  30  19 ... 181 148  53]
 [ 95 185 228 ...   3 229  63]
 [188 134 195 ...  75 112 245]]


### Umbral basado en Varianza

In [37]:


def calculate_threshold_based_on_variance(image):
    # Calcular la frecuencia de cada intensidad de píxel
    unique, counts = np.unique(image, return_counts=True)
    frequencies = dict(zip(unique, counts))
    
    # Calcular la probabilidad de cada intensidad
    total_pixels = image.size
    probabilities = {k: v / total_pixels for k, v in frequencies.items()}
    
    # Inicializar variables para el cálculo del umbral
    best_threshold = None
    max_variance = 0
    
    for threshold in unique:
        # Dividir las intensidades en dos grupos basados en el umbral
        background = {k: probabilities[k] for k in probabilities if k <= threshold}
        foreground = {k: probabilities[k] for k in probabilities if k > threshold}
        
        # Calcular pesos (probabilidades totales) de los fondos y formas
        weight_background = sum(background.values())
        weight_foreground = sum(foreground.values())
        
        # Evitar división por cero
        if weight_background == 0 or weight_foreground == 0:
            continue
        
        # Calcular medias de cada grupo
        mean_background = sum(k * v for k, v in background.items()) / weight_background
        mean_foreground = sum(k * v for k, v in foreground.items()) / weight_foreground
        
        # Calcular varianza entre clases
        variance_between = weight_background * weight_foreground * (mean_background - mean_foreground) ** 2
        
        # Actualizar el mejor umbral si se encuentra una varianza mayor
        if variance_between > max_variance:
            max_variance = variance_between
            best_threshold = threshold
    
    return best_threshold, image_matrix

# Calcular el umbral basado en la varianza
threshold, sample_image_matrix = calculate_threshold_based_on_variance(image_matrix)
print(threshold)
print(sample_image_matrix)
# Aplicar el umbral de valle global a la matriz original
segmented_by_variance = np.where(image_matrix <= threshold, 0, 255)
print(segmented_by_variance)



127
[[  9 125 228 ... 205  88 143]
 [253 134 246 ... 161  95 171]
 [216 224 229 ...  44   9  50]
 ...
 [ 94  30  19 ... 181 148  53]
 [ 95 185 228 ...   3 229  63]
 [188 134 195 ...  75 112 245]]
[[  0   0 255 ... 255   0 255]
 [255 255 255 ... 255   0 255]
 [255 255 255 ...   0   0   0]
 ...
 [  0   0   0 ... 255 255   0]
 [  0 255 255 ...   0 255   0]
 [255 255 255 ...   0   0 255]]


### Umbral basado en entropía

In [38]:
import numpy as np

def calculate_threshold_based_on_entropy(image):
    unique, counts = np.unique(image, return_counts=True)
    total_pixels = image.size
    probabilities = counts / total_pixels
    
    # Calcular la entropía para cada posible umbral
    best_threshold = None
    max_entropy = -np.inf
    
    for threshold in range(unique.min(), unique.max()):
        # Dividir las intensidades en dos grupos basados en el umbral
        background_probs = probabilities[unique <= threshold]
        foreground_probs = probabilities[unique > threshold]
        
        # Evitar divisiones por cero
        if background_probs.size == 0 or foreground_probs.size == 0:
            continue
        
        # Calcular la entropía para el fondo y para el objeto
        background_entropy = -np.sum(background_probs * np.log2(background_probs + np.finfo(float).eps))
        foreground_entropy = -np.sum(foreground_probs * np.log2(foreground_probs + np.finfo(float).eps))
        
        # Calcular la entropía total como la suma de ambas entropías
        total_entropy = background_entropy + foreground_entropy
        
        # Actualizar el mejor umbral si se encuentra una entropía total mayor
        if total_entropy > max_entropy:
            max_entropy = total_entropy
            best_threshold = threshold
    
    return best_threshold

# Calcular el umbral basado en la entropía
threshold_entropy = calculate_threshold_based_on_entropy(image_matrix)

print(threshold_entropy)
# Aplicar el umbral basado en entropía a la matriz original
segmented_by_entropy = np.where(image_matrix <= threshold_entropy, 0, 255)

print(segmented_by_entropy)

76
[[  0 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ...   0   0   0]
 ...
 [255   0   0 ... 255 255   0]
 [255 255 255 ...   0 255   0]
 [255 255 255 ...   0 255 255]]


### Valle global

In [39]:
def find_global_valley_threshold_corrected(image):
    unique, counts = np.unique(image.ravel(), return_counts=True)
    total_pixels = image.size
    
    # Crear el histograma de la imagen
    histogram, bin_edges = np.histogram(image, bins=np.arange(257), density=True)
    
    # Calcular la función de coste para cada umbral potencial
    best_threshold = None
    min_cost = np.inf
    
    for threshold in range(1, 256):
        # Dividir el histograma en dos partes
        hist_background = histogram[:threshold]
        hist_foreground = histogram[threshold:]
        
        # Calcular las probabilidades acumuladas para el fondo y el objeto
        omega_background = np.sum(hist_background)
        omega_foreground = np.sum(hist_foreground)
        
        # Evitar división por cero
        if omega_background == 0 or omega_foreground == 0:
            continue
        
        # Asegurar que las dimensiones de los arrays coinciden para las operaciones
        mu_background = np.sum(hist_background * np.arange(threshold)) / omega_background
        mu_foreground = np.sum(hist_foreground * np.arange(threshold, 256)) / omega_foreground
        
        # Calcular la varianza entre clases (o su inversa como coste)
        variance_between = omega_background * omega_foreground * (mu_background - mu_foreground) ** 2
        
        # Buscar el mínimo de la varianza entre clases (máximo de la inversa)
        if variance_between < min_cost:
            min_cost = variance_between
            best_threshold = threshold
    
    return best_threshold

# Intentar de nuevo calcular el umbral de valle global
threshold_global_valley_corrected = find_global_valley_threshold_corrected(image_matrix)

print(threshold_global_valley_corrected)

# Aplicar el umbral de valle global a la matriz original
segmented_by_global_valley = np.where(image_matrix <= threshold_global_valley_corrected, 0, 255)

print(segmented_by_global_valley)

255
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Vecindarios

In [40]:
def identify_neighborhoods(image, similarity_threshold, proximity=1):
    """
    Identifica vecindarios en una imagen basándose en la similitud de intensidad y la proximidad física.
    
    :param image: Matriz de la imagen en escala de grises.
    :param similarity_threshold: Umbral para la diferencia máxima de intensidad entre píxeles vecinos.
    :param proximity: Distancia máxima considerada para la proximidad física (1 para vecinos inmediatos).
    :return: Matriz con los mismos tamaños que la imagen, donde cada elemento es el ID del vecindario al que pertenece.
    """
    # Inicializar la matriz de vecindarios con ceros (ningún píxel asignado a un vecindario)
    neighborhoods = np.zeros_like(image, dtype=int)
    neighborhood_id = 1  # Iniciar IDs de vecindario
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if neighborhoods[i, j] == 0:  # Si el píxel aún no ha sido asignado a un vecindario
                # Marcar el píxel actual con el ID de vecindario actual y buscar vecinos similares
                search_similar_neighbors(image, neighborhoods, i, j, neighborhood_id, similarity_threshold, proximity)
                neighborhood_id += 1  # Incrementar el ID para el próximo vecindario
    
    return neighborhoods

def search_similar_neighbors(image, neighborhoods, i, j, neighborhood_id, threshold, proximity):
    """
    Busca y asigna vecinos similares al mismo vecindario de un píxel dado.
    
    :param image: Matriz de la imagen en escala de grises.
    :param neighborhoods: Matriz de los vecindarios asignados.
    :param i: Índice de fila del píxel actual.
    :param j: Índice de columna del píxel actual.
    :param neighborhood_id: ID del vecindario para asignar a los vecinos similares encontrados.
    :param threshold: Umbral de similitud para considerar a los vecinos.
    :param proximity: Proximidad para considerar vecinos.
    """
    if i < 0 or i >= image.shape[0] or j < 0 or j >= image.shape[1]:  # Si el índice está fuera de la matriz
        return
    if neighborhoods[i, j] != 0:  # Si el píxel ya ha sido asignado a un vecindario
        return
    
    # Marcar el píxel actual con el ID de vecindario
    neighborhoods[i, j] = neighborhood_id
    
    # Buscar vecinos en la proximidad definida
    for di in range(-proximity, proximity+1):
        for dj in range(-proximity, proximity+1):
            if 0 <= i+di < image.shape[0] and 0 <= j+dj < image.shape[1]:  # Asegurarse de que el vecino esté dentro de la imagen
                if abs(image[i, j] - image[i+di, j+dj]) <= threshold:  # Si el vecino es similar
                    search_similar_neighbors(image, neighborhoods, i+di, j+dj, neighborhood_id, threshold, proximity)  # Asignar vecino al mismo vecindario

# Vamos a probar este enfoque en una matriz de ejemplo
np.random.seed(4)  # Generar una nueva matriz de imagen para este ejemplo
image_matrix_5 = np.random.randint(0, 256, size=(10, 10))
neighborhoods = identify_neighborhoods(image_matrix_5, similarity_threshold=20, proximity=1)

neighborhoods


array([[ 1,  2,  2,  2,  3,  3,  3,  4,  5,  6],
       [ 7,  8,  3,  3,  3,  9,  4,  5, 10, 11],
       [12,  7,  7,  3, 13,  9, 14, 15, 11, 16],
       [17, 18, 19, 13, 13,  9, 11, 11, 20, 16],
       [17, 21, 21, 19, 22, 11, 11, 23, 11, 11],
       [24, 21, 25, 26, 11, 27, 28, 28, 29, 30],
       [31, 32, 26, 33, 11, 34, 29, 29, 35, 29],
       [32, 32, 36, 37, 34, 38, 29, 29, 29, 35],
       [39, 40, 37, 41, 34, 34, 34, 29, 29, 29],
       [42, 42, 34, 34, 43, 34, 44, 45, 29, 46]])